# Objective

Predict if a property is a good investment based on financial and physical attributes.

<b>Target:</b>

A binary variable indicating good (1) or bad (0) investment. Define this based on criteria such as a high rent-to-price ratio or a favorable market estimate compared to the listed price.


In [284]:
import pandas as pd

# Data exploration


In [285]:
real_estate_data = pd.read_csv("./data/real_estate_data.csv")
real_estate_data.describe()

# Find any missing data in the dataset.
missing_values = real_estate_data.isnull().sum()

missing_values

State                0
City                 0
Street               0
Zipcode              0
Bedroom             14
Bathroom            34
Area                 0
PPSq                 0
LotArea            902
MarketEstimate    7236
RentEstimate      5976
Latitude             0
Longitude            0
ListedPrice          0
dtype: int64

In [286]:
# See how much data there is with no null values. This can help us determine if we can simply remove all of the nulls from the dataset.

len(real_estate_data.dropna()), real_estate_data.dropna().isnull().sum()

(14853,
 State             0
 City              0
 Street            0
 Zipcode           0
 Bedroom           0
 Bathroom          0
 Area              0
 PPSq              0
 LotArea           0
 MarketEstimate    0
 RentEstimate      0
 Latitude          0
 Longitude         0
 ListedPrice       0
 dtype: int64)

In [287]:
# Get the average difference of List to Estimate by state.

no_null_market_estimate = real_estate_data[real_estate_data["MarketEstimate"].notnull()]

avg_market_minus_list = (
    real_estate_data["MarketEstimate"] - real_estate_data["ListedPrice"]
).median()
avg_market_minus_list

-1600.0

# Data preprocessing


In [288]:
# Remove all rows without a Rent Estimate as this value is vital to my analysis.

real_estate_data = real_estate_data[real_estate_data["RentEstimate"].notnull()]

len(real_estate_data), real_estate_data.isnull().sum()

(16705,
 State                0
 City                 0
 Street               0
 Zipcode              0
 Bedroom              9
 Bathroom            24
 Area                 0
 PPSq                 0
 LotArea            611
 MarketEstimate    1308
 RentEstimate         0
 Latitude             0
 Longitude            0
 ListedPrice          0
 dtype: int64)

In [289]:
# Update market estimates with the average difference between
for index, row in real_estate_data.iterrows():
    if pd.isna(row["MarketEstimate"]):
        real_estate_data.at[index, "MarketEstimate"] = (
            row["ListedPrice"] + avg_market_minus_list
        )

real_estate_data.isnull().sum()

State               0
City                0
Street              0
Zipcode             0
Bedroom             9
Bathroom           24
Area                0
PPSq                0
LotArea           611
MarketEstimate      0
RentEstimate        0
Latitude            0
Longitude           0
ListedPrice         0
dtype: int64

In [290]:
# Populate bed, bath, and lot area based on the median area for the given state.


def get_median_for_row(row: pd.Series, df: pd.DataFrame, area_offset=200):
    area = row["Area"]
    filtered_df = df[df["State"].eq(row["State"])].query(
        f"Area <= {area + area_offset} or Area >= {area - area_offset}"
    )
    median_bed = filtered_df["Bedroom"].median()
    median_bath = filtered_df["Bathroom"].median()
    median_lot = filtered_df["LotArea"].median()
    return median_bed, median_bath, median_lot


for index, row in real_estate_data.iterrows():
    if pd.isna(row["Bedroom"]) or pd.isna(row["Bathroom"]) or pd.isna(row["LotArea"]):
        if pd.isna(row["Bedroom"]):
            median_bed, _, _ = get_median_for_row(row, real_estate_data)
            real_estate_data.at[index, "Bedroom"] = median_bed
        if pd.isna(row["Bathroom"]):
            _, median_bath, _ = get_median_for_row(row, real_estate_data)
            real_estate_data.at[index, "Bathroom"] = median_bath
        if pd.isna(row["LotArea"]):
            _, _, median_lot = get_median_for_row(row, real_estate_data)
            real_estate_data.at[index, "LotArea"] = median_lot

real_estate_data.isnull().sum()

State             0
City              0
Street            0
Zipcode           0
Bedroom           0
Bathroom          0
Area              0
PPSq              0
LotArea           0
MarketEstimate    0
RentEstimate      0
Latitude          0
Longitude         0
ListedPrice       0
dtype: int64

In [291]:
from src.const import property_tax_rates

# Calculating the additional features:

real_estate_data["AnnualPropertyTaxEstimate"] = real_estate_data[
    "MarketEstimate"
] * real_estate_data["State"].map(property_tax_rates)


# Estimated Monthly Mortgage - assuming a 30-year fixed mortgage at 6% interest rate.
interest_rate = 0.06 / 12
loan_term = 30 * 12  # 30 years fixed rate loan.
real_estate_data["MonthlyMortgageEstimate"] = (
    real_estate_data["ListedPrice"] * interest_rate * (1 + interest_rate) ** loan_term
) / ((1 + interest_rate) ** loan_term - 1)

real_estate_data[
    [
        "ListedPrice",
        "MarketEstimate",
        "RentEstimate",
        "AnnualPropertyTaxEstimate",
        "MonthlyMortgageEstimate",
    ]
].head()

,ListedPrice,MarketEstimate,RentEstimate,AnnualPropertyTaxEstimate,MonthlyMortgageEstimate
0,239900.0,240600.0,1599.0,986.46,1438.321710
3,335000.0,336200.0,1932.0,1378.42,2008.494259
4,250000.0,222700.0,1679.0,913.07,1498.876313
5,151000.0,150500.0,1385.0,617.05,905.321293
6,239000.0,238400.0,2125.0,977.44,1432.925755


In [292]:
# For this exercise I am going to populate the "GoodInvestment" field with 1 when the Rent price exceeds the annual cost of the home.

real_estate_data["AnnualCost"] = (
    real_estate_data["MonthlyMortgageEstimate"] * 12
) + real_estate_data["AnnualPropertyTaxEstimate"]
real_estate_data["AnnualIncome"] = real_estate_data["RentEstimate"] * 12

real_estate_data["GoodInvestment"] = real_estate_data.apply(
    lambda row: 1 if row["AnnualIncome"] > row["AnnualCost"] else 0, axis=1
)

len(real_estate_data), real_estate_data["GoodInvestment"].value_counts()

(16705,
 GoodInvestment
 0    10523
 1     6182
 Name: count, dtype: int64)

In [293]:
X = real_estate_data.drop(
    [
        "GoodInvestment",
        "AnnualCost",
        "AnnualIncome",
        "RentEstimate",
        "MonthlyMortgageEstimate",
        "AnnualPropertyTaxEstimate",
    ],
    axis=1,
)

y = real_estate_data["GoodInvestment"]

# Proceed with splitting the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

f"Training set size: {X_train.shape[0]}", f"Testing set size: {X_test.shape[0]}"

('Training set size: 13364', 'Testing set size: 3341')

# Decision Tree
